In [1]:
import torch.nn   as nn
import GM_library as GML
import numpy      as np
import torch

from os                   import path, listdir
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps, which can be different
n_denoising_steps = 10

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Define box shape
L = [20, 20, 20]

# Target to generate new crystals
target = 'GM_EPA'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_EMP'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [3]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset    = torch.load(dataset_name_std)
parameters = torch.load(parameters_name_std)

# Assigning parameters accordingly
target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters

# Defining target factor
target_factor = target_std / scale

In [4]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(144.71099853515625, 50.02598190307617)

# Loading the model

In [5]:
# Determine number of features in dataset
n_features = dataset[0].num_node_features

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = GML.eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(4, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 4)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=4, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


# Generating new cystals

In [75]:
import importlib
importlib.reload(GML)

<module 'GM_library' from '/Users/cibran/Work/UPC/GenerativeModels/GM_library.py'>

In [6]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Diffuse the graph with some noise
        diffused_graph = GML.get_random_graph(n_nodes, n_features)  # CHECK THIS FUNCTION

        # Denoise the diffused graph
        denoised_graph = diffused_graph.clone()
        for t in range(n_denoising_steps):
            
            # Perform a single forward pass for predicting node features
            out_x = node_model(diffused_graph.x, 
                               diffused_graph.edge_index,
                               diffused_graph.edge_attr)

            # Define x_i and x_j as features of every corresponding pair of nodes (same order than attributes)
            x_i = diffused_graph.x[diffused_graph.edge_index[0]]
            x_j = diffused_graph.x[diffused_graph.edge_index[1]]

            # Perform a single forward pass for predicting edge attributes
            out_attr = edge_model(x_i, x_j)

            # Construct noise graph
            noise_graph = Data(x=out_x, edge_index=diffused_graph.edge_index, edge_attr=out_attr.ravel())

            # Denoise the graph with the predicted noise
            denoised_graph = GML.denoising_step(denoised_graph, noise_graph, t, n_denoising_steps)

        # Append generated graph
        predicted_dataset.append(denoised_graph)

In [7]:
predicted_dataset

[Data(x=[58, 4], edge_index=[2, 1710], edge_attr=[1710]),
 Data(x=[128, 4], edge_index=[2, 8255], edge_attr=[8255]),
 Data(x=[14, 4], edge_index=[2, 104], edge_attr=[104]),
 Data(x=[52, 4], edge_index=[2, 1377], edge_attr=[1377]),
 Data(x=[108, 4], edge_index=[2, 5885], edge_attr=[5885]),
 Data(x=[141, 4], edge_index=[2, 10010], edge_attr=[10010]),
 Data(x=[143, 4], edge_index=[2, 10295], edge_attr=[10295]),
 Data(x=[114, 4], edge_index=[2, 6554], edge_attr=[6554]),
 Data(x=[121, 4], edge_index=[2, 7380], edge_attr=[7380]),
 Data(x=[94, 4], edge_index=[2, 4464], edge_attr=[4464])]

In [20]:
import importlib
importlib.reload(GML)

<module 'GM_library' from '/Users/cibran/Work/UPC/GenerativeModels/GM_library.py'>

In [29]:
edge_indexes = graph_raw.edge_index.detach().cpu().numpy()

In [51]:
index0 = 1
index1 = 8

edge_indexes0 = edge_indexes[0]
edge_indexes1 = edge_indexes[1]

for i in range(len(edge_indexes0)):
    if ((edge_indexes0[i] == index0) and (edge_indexes1[i] == index1)) or ((edge_indexes0[i] == index1) and (edge_indexes1[i] == index0)):
        print(i)
        break

print(edge_indexes[:, i]) 

65
[1 8]


In [52]:


print(edge_indexes[:, i]) 

65
[1 8]


array([0, 2])

In [38]:
edge_indexes[0]

array([ 0,  0,  0, ..., 55, 56, 56])

In [40]:
np.where((edge_indexes[0] == index0) and (edge_indexes[1] == index1))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [30]:
edge_indexes

array([[ 0,  0,  0, ..., 55, 56, 56],
       [ 0,  1,  2, ..., 57, 56, 57]])

In [19]:
graph_raw      = predicted_dataset[0]
graph_discrete = GML.discretize_graph(graph_raw)

In [22]:
node_loss, edge_loss = GML.get_graph_losses(graph_raw, graph_discrete)
node_loss, edge_loss

(tensor(20.2963), tensor(0.))